# Coding with LLMs 301


In [106]:
# Ooodles of Imports 
import os
import json
from icecream import ic
import typer
from rich.console import Console
from rich import print
from typing import List
from pydantic import BaseModel
from loguru import logger
import pudb
from typing_extensions import Annotated

console = Console()
app = typer.Typer()
from langchain.llms import GPT4All
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from typing import Any, Optional
from langchain.output_parsers.openai_functions import OutputFunctionsParser
from langchain.schema import FunctionMessage


from langchain.schema import (
    Generation,
    OutputParserException,
)
import  mercury as mr

In [72]:
# Useful helpers
def model_to_openai_function(cls):
    return {"name": cls.__name__, "parameters": cls.model_json_schema()}
class JsonOutputFunctionsParser2(OutputFunctionsParser):
    """Parse an output as the Json object."""

    def parse_result(self, result: List[Generation]) -> Any:
        function_call_info = super().parse_result(result)
        if self.args_only:
            try:
                # Waiting for this to merge upstream
                return json.loads(function_call_info, strict=False)
            except (json.JSONDecodeError, TypeError) as exc:
                raise OutputParserException(
                    f"Could not parse function call data: {exc}"
                )
        function_call_info["arguments"] = json.loads(function_call_info["arguments"])
        return function_call_info


from IPython.display import display, HTML
def print_line():
    display(HTML('<hr>'))

### Langchain - Super cool, we'll use it, but not our focus

### Compilers - Prompt Templates


In [75]:
# Everyone wants to be a comedian
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
joke_prompt = ChatPromptTemplate.from_template("tell me {count} jokes about {topic}")
print(joke_prompt.messages)

[
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=['count', 'topic'],
            output_parser=None,
            partial_variables={},
            template='tell me {count} jokes about {topic}',
            template_format='f-string',
            validate_template=True
        ),
        additional_kwargs={}
    )
]

In [98]:
# Compile the program for a CPU

model = ChatOpenAI()
chain = joke_prompt | model

# Run it => Invoke()

topic = "Software Engineers"
count = 2
result = chain.invoke({"topic":topic, "count":count})
print (result)
print_line()
print (result.content)


AIMessage(
    content='Sure, here are two jokes about software engineers:\n\n1. Why do software engineers always mix up 
Christmas and Halloween?\n   Because Oct 31 == Dec 25!\n\n2. Why did the software engineer go broke?\n   Because he
lost his domain in a bet!',
    additional_kwargs={},
    example=False
)

Sure, here are two jokes about software engineers:

1. Why do software engineers always mix up Christmas and Halloween?
   Because Oct 31 == Dec 25!

2. Why did the software engineer go broke?
   Because he lost his domain in a bet!

### The Operating System - Functions 

In [81]:
# The rain in spain
# Tell the model the "OS" supports getting the weather


# define a callable function
class GetWeather(BaseModel):
    City: str

get_weather = model_to_openai_function(GetWeather)

weather_prompt_template = "What's the weather in {place}"
model = ChatOpenAI()
weather_prompt = ChatPromptTemplate.from_template(weather_prompt_template)

chain = (
    weather_prompt
    | model.bind(functions=[get_weather]) # tell model we can call it.
)


response = chain.invoke({"place":"Spain"})
print(response)

AIMessage(
    content='',
    additional_kwargs={'function_call': {'name': 'GetWeather', 'arguments': '{\n  "City": "Spain"\n}'}},
    example=False
)

In [96]:
# Woah  - Did you see  the bug? 
model = ChatOpenAI(model="gpt-4")

# Do "some more programming"
weather_prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template("When an API takes a city, infer an appropritiate city"),
            HumanMessagePromptTemplate.from_template(weather_prompt_template),
        ]
    )
chain = (
    weather_prompt
    | model.bind(functions=[get_weather])
)
response = chain.invoke({"place":"Spain"})
print("Output")
print(response)

Output

AIMessage(
    content='',
    additional_kwargs={'function_call': {'name': 'GetWeather', 'arguments': '{\n"City": "Madrid"\n}'}},
    example=False
)

In [100]:
# Back to our functions

weather_with_data = weather_prompt.copy()

# Update prompt with AI's desire to call a function
weather_with_data.append(response)

# Need to make tomorrow's cut, just stamp this please :) 
# Will come back and make a dispatcher and call actual functions

weather_with_data.append(FunctionMessage(name="GetWeather",content="5 degrees and rainy"))

print (weather_with_data)


chain = (
    weather_with_data
    | model.bind(functions=[get_weather]) # tell model we can call it.
)

response = chain.invoke({"place":"Spain"})
print_line()
print(response)

ChatPromptTemplate(
    input_variables=['place'],
    output_parser=None,
    partial_variables={},
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=[],
                output_parser=None,
                partial_variables={},
                template='When an API takes a city, infer an appropritiate city',
                template_format='f-string',
                validate_template=True
            ),
            additional_kwargs={}
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['place'],
                output_parser=None,
                partial_variables={},
                template="What's the weather in {place}",
                template_format='f-string',
                validate_template=True
            ),
            additional_kwargs={}
        ),
        AIMessage(
            content='',
            additional_kwargs={'function_call': {'name': 'GetWeather', 'arguments': '{\n"City": "Madrid"\n}'}},
            example=False
        ),
        FunctionMessage(content='5 degrees and rainy', additional_kwargs={}, name='GetWeather'),
        AIMessage(
            content='The weather in Madrid, Spain is currently 5 degrees and rainy.',
            additional_kwargs={},
            example=False
        ),
        FunctionMessage(content='5 degrees and rainy', additional_kwargs={}, name='GetWeather'),
        AIMessage(
            content='The weather in Spain varies depending on the city you are referring to. Could you please 
specify which city in Spain you are interested in?',
            additional_kwargs={},
            example=False
        ),
        FunctionMessage(content='5 degrees and rainy', additional_kwargs={}, name='GetWeather')
    ]
)

AIMessage(
    content='The weather in Madrid, Spain is currently 5 degrees and rainy.',
    additional_kwargs={},
    example=False
)

In [5]:
# Innovate - Why do we seperate view from model?

class Joke(BaseModel):
    setup: str
    punch_line: str
    reason_joke_is_funny: str

class GetJokes(BaseModel):
    count: int
    jokes: List[Joke]

get_jokes = model_to_openai_function(GetJokes)

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me {count} jokes about {topic}")
chain = (
    prompt
    | model.bind(functions=[get_jokes])
    | JsonOutputFunctionsParser2()
)
print(prompt.messages)
response = chain.invoke({"topic": topic, "count": count})
print("Output")
print(response)

NameError: name 'model_to_openai_function' is not defined

In [104]:
# Innovate - What's better then doing math with a calculator? 


solve_math_with_python = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Write code to solve the users problem. the last line of the python  program should print the answer. Do not use sympy"
        ),
        HumanMessagePromptTemplate.from_template("What is the 217th prime"),
    ]
)

class ExecutePythonCode(BaseModel):
    valid_python: str
    code_explanation: str


model = ChatOpenAI(model="gpt-4").bind(
    function_call={"name": "ExecutePythonCode"},  # tell gpt to use this model
    functions=[model_to_openai_function(ExecutePythonCode)],
)


# JsonOutputFunctionParser2 == PrettyPrintOutput

chain = solve_math_with_python | model | JsonOutputFunctionsParser2()
response = chain.invoke({})

print(response["code_explanation"])
print_line()

valid_python = response["valid_python"]
print(valid_python)




The given Python code first defines a function to check if a number is prime. It then defines a function to get the
nth prime number. It finally prints the 217th prime number.

def is_prime(n):
    if n <= 1:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    i = 3
    while i * i <= n:
        if n % i:
            i += 2
        else:
            return False
    return True


def nth_prime(n):
    count = 0
    num = 1
    while count < n:
        num += 1
        if is_prime(num):
            count += 1
    return num

print(nth_prime(217))

In [107]:
# Ugh ... are you sure you want to do this??
print_li
# input("Are you sure you want to run this code??")
exec(valid_python)

NameError: name 'print_rule' is not defined